In [5]:
import sys
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# import local modules
sys.path.append('../src')
from Fishnet import Fishnet
from ImageExporter import ImageExporter
from ImageCorrector import ImageCorrector
from ImageProcessor import ImageProcessor

# update files automatically
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fc = Fishnet.load("../Gis/Fishnet/fishnet_quarter_mile_completed.pkl")

In [3]:
# Compute Urbanization Rate
for year in [2017, 2018, 2019, 2020, 2021, 2022]:
    fc.compute_difference(f'MeanPixel_{year}', f'MeanPixel_{year-1}', filtered=True, normalize = True)

fc.filtered_fishnet.rename(columns={'MeanPixel_2022-MeanPixel_2021': 'UrbanizationRate2022'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2021-MeanPixel_2020': 'UrbanizationRate2021'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2020-MeanPixel_2019': 'UrbanizationRate2020'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2019-MeanPixel_2018': 'UrbanizationRate2019'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2018-MeanPixel_2017': 'UrbanizationRate2018'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2017-MeanPixel_2016': 'UrbanizationRate2017'}, inplace=True)
fc.filtered_fishnet.drop(columns=['MeanPixel_2021-MeanPixel_2016'], inplace=True)

fc.filtered_fishnet.head(3)

,id,geometry,batch_id,neighbors,ImageCoordinates,Width,Height,MeanPixel_2016,Entropy_2016,MeanPixel_2017,...,MeanPixel_2021,Entropy_2021,MeanPixel_2022,Entropy_2022,UrbanizationRate2017,UrbanizationRate2018,UrbanizationRate2019,UrbanizationRate2020,UrbanizationRate2021,UrbanizationRate2022
6060154,6060154.0,"POLYGON ((-95.79985 29.79208, -95.79985 29.795...",23743.0,"[6056884, 6056885, 6056886, 6060153, 6060155, ...","(403, 524, 448, 564)",45.0,40.0,181.191667,0.868003,197.341667,...,233.041667,0.423354,237.008333,0.367993,0.063333,0.097778,-0.010556,0.025000,0.027778,0.015556
6060155,6060155.0,"POLYGON ((-95.79583 29.79208, -95.79583 29.795...",23743.0,"[6056885, 6056886, 6056887, 6060154, 6060156, ...","(448, 524, 492, 564)",44.0,40.0,204.724432,0.716210,213.272727,...,237.613636,0.359102,241.815341,0.293594,0.033523,0.063068,-0.003409,0.009659,0.026136,0.016477
6060156,6060156.0,"POLYGON ((-95.79181 29.79208, -95.79181 29.795...",23743.0,"[6056886, 6056887, 6056888, 6060155, 6060157, ...","(492, 524, 537, 564)",45.0,40.0,140.250000,0.992774,133.025000,...,168.583333,0.923751,168.583333,0.923751,-0.028333,0.014444,0.041667,0.005556,0.077778,0.000000


In [6]:
vars = ["id", "batch_id", "MeanPixel_2016", "MeanPixel_2017", "MeanPixel_2018", "MeanPixel_2019", "MeanPixel_2020", "MeanPixel_2021"]
data = fc.filtered_fishnet[vars]
data.rename(columns={"id": "tile_id"}, inplace=True)
data["tile_id"] = data["tile_id"].astype(int)
data["batch_id"] = data["batch_id"].astype(int)

# Pivot wide to long
data = data.melt(id_vars=["tile_id", "batch_id"], var_name="year", value_name="urbanization")
data["year"] = data["year"].str[-4:]
data.head()

,tile_id,batch_id,year,urbanization
0,6060154,23743,2016,181.191667
1,6060155,23743,2016,204.724432
2,6060156,23743,2016,140.250000
3,6060157,23743,2016,114.041667
4,6060158,23743,2016,197.200000


In [ ]:
# I need to rebuild a large matrix for each year with the urbanization rate for each tile

In [9]:
print(data['tile_id'])

0         6060154
1         6060155
2         6060156
3         6060157
4         6060158
           ...   
135481    6439546
135482    6439547
135483    6439548
135484    6439549
135485    6439550
Name: tile_id, Length: 135486, dtype: int64


In [16]:
data2016 = data[data["year"]== '2016']


(22581,)

In [ ]:
data2016["urbanization"].to_numpy().shape 

In [18]:
fc.fishnet_cols

3269

In [19]:
fc.fishnet_rows

2948

In [20]:
2948*3269

9637012